### Getting Started with you Notebook

#### 1. Select your interpreter
- [Press 'Cmd + Shift + P' and enter 'Select Notebook Kernel'](https://code.visualstudio.com/docs/datascience/jupyter-notebooks) or click 'Select Kernel' on the top right ↗️↗️↗️  

<img src="../docs/images/select_kernel.png" alt="drawing" width="400"/>  
  



- Choose 'Python Environments'  

<img src="../docs/images/python_envs.png" alt="drawing" width="400"/>  

- Select the default recommended option (Note: it *should* be stored in your project .pixi directory)  
  
  
<img src="../docs/images/select_default.png" alt="drawing" width="400"/>



#### 2. Test your imports
- The cells below ensures your local package is imported correctly as well as pixi dependencies 

⚠️ Note: If you're making updates to your local code, you'll need to restart the kernel to get around object caching

In [ ]:
# Check local package imports correctly
from  src.ds_ticat.ModelManager import ModelManager

PROJECT_ROOT = "/home/j0sch111/github/ds-ticat"

print("Creating ModelManager instance...")
model_manager = ModelManager(project_root=PROJECT_ROOT)

# Validate setup
model_manager.validate_setup()

## Optional: Experiment Tracking

In [ ]:
from src_labs.experiment_tracking.meta_tracker import MetaTrackerFactory, TrackerConfig, TrackerType

PROJECT_NAME = "Red-Zebra"
configs = [
    TrackerConfig(TrackerType.WANDB),                          # Uncomment to add more trackers
    #TrackerConfig(TrackerType.NEPTUNE, project=PROJECT_NAME),
    #TrackerConfig(TrackerType.COMET, workspace="ryanatsevwork-gmail-com", project_name=PROJECT_NAME),
    #TrackerConfig(TrackerType.MLFLOW, tracking_uri="http://127.0.0.1:5000")
]

meta_tracker = MetaTrackerFactory.create_meta_tracker(configs)

# Use meta_tracker as before
try:
    meta_tracker.start_run(PROJECT_NAME, tags={"version": "1.0"})
    print("MetaTracker run started successfully")
except Exception as e:
    print(f"Error starting MetaTracker run: {str(e)}")
    print("Some trackers may have failed to initialize. Check individual tracker logs for details.")


In [ ]:

# Train the model
model_training_manager = ModelManager(project_root=PROJECT_ROOT, experiment_tracker=meta_tracker)
model_training_manager.train()

# Make predictions
print("\nMaking predictions...")
positive_text = "This product is fantastic!"
negative_text = "I'm disappointed with this service."

label, confidence = model_training_manager.predict(positive_text)
print(f"Prediction for '{positive_text}': {label} (Confidence: {confidence:.2f})")

label, confidence = model_training_manager.predict(negative_text)
print(f"Prediction for '{negative_text}': {label} (Confidence: {confidence:.2f})")

# Print project info
print("\nProject Information:")
project_info = model_training_manager.get_project_info()
for key, value in project_info.items():
    print(f"{key}: {value}")

## Containerizing your model

The following takes the demo model code and packages so that we can run inference with it. One, by running a lambda locally, the other via sagemaker. These can subsequently be deployed to AWS quickly as we start to prototype with models

In [ ]:
from infrastructure.containers.utils.container_client_utils import ContainerClientUtils

process = ContainerClientUtils.start_model_containers()

#### Lambda

This is defined in ./infrastructure/lambda and starts a container that conforms to the AWS lambda API. It's not suitable for larger GPU based models but could be a suitable solution for cheaply deploying small models with variable traffic

In [ ]:
print("Calling Lambda Container:")
lambda_text_input = "thats awesome! I love it!"
lambda_result = ContainerClientUtils.call_lambda_container(lambda_text_input)

#### Sagemaker

Much more robust model hosting platform that offers full GPU support and combining of multiple models per endpoint. Downsides are expense and always on architecture. Defined in [../infrastructure/sagemaker](vscode://file/Users/ryanpasecky/workplace/ds-meta/infrastructure/sagemaker))

In [ ]:
print("\nCalling SageMaker Container:")
sagemaker_text_input = "thats awesome! no way!"
sagemaker_result = ContainerClientUtils.call_sagemaker_container(sagemaker_text_input)

#### Integration testing

We've also configured an integration testing suite which can be used to test our code from end to end. For now
it's defined using the popular pytest, and run from a third container defined in [/infrastructure/containers/Dockerfile_IntegrationTests/](vscode://file//home/j0sch111/github/ds-ticat/test/integration/)

In [ ]:
import subprocess
DOCKER_DIR = "/home/j0sch111/github/ds-ticat/infrastructure/containers"

process = subprocess.Popen(["docker", "compose", "--project-directory", DOCKER_DIR, "up"])

In [ ]:
print("Hello")